In [ ]:
!pip uninstall -y numpy pandas tensorflow
!pip install numpy==1.23.5 pandas==1.5.3 tensorflow==2.10.0 matplotlib pillow tqdm scikit-learn

In [ ]:
# 匯入套件
import os
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import tensorflow as tf

# 設定隨機種子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# Global parameters
IMAGE_SIZE = 512
BATCH_SIZE = 8
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

# 顯示詳細 GPU 訊息
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


In [ ]:
!pip install kaggle --upgrade


In [ ]:
from tensorflow.keras import mixed_precision

# 啟用 mixed precision
mixed_precision.set_global_policy('mixed_float16')
print("✅ Mixed Precision enabled: ", mixed_precision.global_policy())


In [ ]:
import os

# 建立資料夾
os.makedirs('/root/.kaggle', exist_ok=True)

# 假設 kaggle.json 已經上傳到當前目錄
!mv kaggle.json /root/.kaggle/

# 設定權限
!chmod 600 /root/.kaggle/kaggle.json
print("✅ kaggle API 已設定完成")

In [ ]:
!kaggle datasets list -s deepfashion

In [ ]:
!kaggle datasets download -d hserdaraltan/deepfashion-inshop-clothes-retrieval-adjusted -p /workspace/data
!unzip -q /workspace/data/deepfashion-inshop-clothes-retrieval-adjusted.zip -d /workspace/data/deepfashion

In [ ]:
# 資料載入與完整性驗證 

print("🔍 載入並驗證影像和遮罩...")
print("="*70)

# Define the paths for images and masks using the path from the previous cell output
# Assuming the images and masks are located in 'images' and 'masks' subdirectories within the downloaded path
path = '/workspace/data/deepfashion/'
IMAGES_PATH = os.path.join(path, 'images/')
MASKS_PATH = os.path.join(path, 'masks/')

# 假設你已經將資料集解壓縮到 /workspace/data/deepfashion
path = '/workspace/data/deepfashion/'

def load_and_validate_images(path, file_type="images"):
    """
    載入並驗證影像檔案

    檢查項目：
    1. 檔案是否可讀取
    2. 尺寸是否合理（> 50x50）
    3. 是否為空白影像（標準差 < 1）
    """
    print(f"\n📂 載入 {file_type}...")

    # 取得所有檔案
    files = glob.glob(path + '*')
    files.sort()

    valid_files = []
    corrupted_files = []
    stats = {
        'total': len(files),
        'valid': 0,
        'corrupted': 0,
        'too_small': 0,
        'blank': 0,
        'read_error': 0
    }

    # 驗證每個檔案
    for file_path in tqdm(files, desc=f"Validating {file_type}"):
        try:
            # 開啟並驗證影像
            img = Image.open(file_path)
            img.verify()  # 驗證檔案完整性

            # 重新開啟以讀取資料（verify後需要重新開啟）
            img = Image.open(file_path)
            width, height = img.size

            # 檢查尺寸
            if width < 50 or height < 50:
                corrupted_files.append((file_path, "too_small"))
                stats['too_small'] += 1
                continue

            # 檢查是否為空白影像
            img_array = np.array(img)
            if img_array.std() < 1:
                corrupted_files.append((file_path, "blank"))
                stats['blank'] += 1
                continue

            # 通過所有檢查
            valid_files.append(file_path)
            stats['valid'] += 1

        except Exception as e:
            corrupted_files.append((file_path, f"error: {str(e)[:30]}"))
            stats['read_error'] += 1

    stats['corrupted'] = stats['too_small'] + stats['blank'] + stats['read_error']

    return valid_files, corrupted_files, stats

# 驗證影像
valid_images, corrupted_images, image_stats = load_and_validate_images(
    IMAGES_PATH, "images"
)

# 驗證遮罩
valid_masks, corrupted_masks, mask_stats = load_and_validate_images(
    MASKS_PATH, "masks"
)

# 顯示統計結果
print("\n" + "="*70)
print("📊 驗證結果摘要:")
print("="*70)

print("\n🖼️ 影像檔案:")
print(f"  總數量: {image_stats['total']}")
# Add a check to prevent division by zero
if image_stats['total'] > 0:
    print(f"  ✅ 有效: {image_stats['valid']} ({image_stats['valid']/image_stats['total']*100:.1f}%)")
    print(f"  ❌ 損壞: {image_stats['corrupted']} ({image_stats['corrupted']/image_stats['total']*100:.1f}%)")
else:
    print("  ✅ 有效: 0 (0.0%)")
    print("  ❌ 損壞: 0 (0.0%)")

if image_stats['corrupted'] > 0:
    print(f"     - 尺寸過小: {image_stats['too_small']}")
    print(f"     - 空白影像: {image_stats['blank']}")
    print(f"     - 讀取錯誤: {image_stats['read_error']}")

print("\n🎭 遮罩檔案:")
print(f"  總數量: {mask_stats['total']}")
# Add a check to prevent division by zero
if mask_stats['total'] > 0:
    print(f"  ✅ 有效: {mask_stats['valid']} ({mask_stats['valid']/mask_stats['total']*100:.1f}%)")
    print(f"  ❌ 損壞: {mask_stats['corrupted']} ({mask_stats['corrupted']/mask_stats['total']*100:.1f}%)")
else:
    print("  ✅ 有效: 0 (0.0%)")
    print("  ❌ 損壞: 0 (0.0%)")

if mask_stats['corrupted'] > 0:
    print(f"     - 尺寸過小: {mask_stats['too_small']}")
    print(f"     - 空白影像: {mask_stats['blank']}")
    print(f"     - 讀取錯誤: {mask_stats['read_error']}")

# 顯示損壞檔案範例
if len(corrupted_images) > 0:
    print("\n⚠️ 損壞的影像檔案範例（前5個）:")
    for i, (file_path, reason) in enumerate(corrupted_images[:5]):
        print(f"  {i+1}. {os.path.basename(file_path)} - {reason}")

if len(corrupted_masks) > 0:
    print("\n⚠️ 損壞的遮罩檔案範例（前5個）:")
    for i, (file_path, reason) in enumerate(corrupted_masks[:5]):
        print(f"  {i+1}. {os.path.basename(file_path)} - {reason}")

# 視覺化驗證結果
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 影像統計
labels_img = ['有效', '尺寸過小', '空白', '讀取錯誤']
sizes_img = [image_stats['valid'], image_stats['too_small'],
             image_stats['blank'], image_stats['read_error']]
colors_img = ['#4CAF50', '#FFC107', '#FF9800', '#F44336']
# Add a check to prevent plotting when there are no images
if image_stats['total'] > 0:
    axes[0].pie(sizes_img, labels=labels_img, colors=colors_img, autopct='%1.1f%%', startangle=90)
axes[0].set_title('影像檔案狀態分佈')

# 遮罩統計
sizes_mask = [mask_stats['valid'], mask_stats['too_small'],
              mask_stats['blank'], mask_stats['read_error']]
colors_mask = ['#4CAF50', '#FFC107', '#FF9800', '#F44336'] # Define colors_mask
# Add a check to prevent plotting when there are no masks
if mask_stats['total'] > 0:
    axes[1].pie(sizes_mask, labels=labels_img, colors=colors_mask, autopct='%1.1f%%', startangle=90)
axes[1].set_title('遮罩檔案狀態分佈')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print(f"✅ 驗證完成！共找到 {len(valid_images)} 個有效影像和 {len(valid_masks)} 個有效遮罩")
print("="*70)

In [ ]:
!apt-get update -qq
!apt-get install fonts-noto-cjk -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Configure matplotlib to use the Noto Sans CJK font
for font in fm.findSystemFonts(fontpaths=['/usr/share/fonts/opentype/noto/']):
    if 'NotoSansCJK' in font:
        fm.fontManager.addfont(font)

plt.rcParams['font.family'] = 'Noto Sans CJK JP'
plt.rcParams['axes.unicode_minus'] = False # Allow negative signs to be displayed

print("✅ Noto Sans CJK font installed and configured.")

In [ ]:
# 影像-遮罩配對驗證
print("🔗 驗證影像和遮罩的配對關係...")
print("="*70)

def verify_image_mask_pairs(image_paths, mask_paths):
    """
    驗證影像和遮罩是否正確配對

    檢查：
    1. 檔名是否一致（去除副檔名）
    2. 數量是否相同
    """
    print("\n🔍 檢查配對關係...")

    # 建立檔名到路徑的映射
    image_dict = {}
    for img_path in image_paths:
        filename = os.path.splitext(os.path.basename(img_path))[0]
        image_dict[filename] = img_path

    mask_dict = {}
    for mask_path in mask_paths:
        filename = os.path.splitext(os.path.basename(mask_path))[0]
        mask_dict[filename] = mask_path

    # 找出配對和未配對的檔案
    image_names = set(image_dict.keys())
    mask_names = set(mask_dict.keys())

    # 完美配對的檔案
    paired_names = image_names & mask_names

    # 只有影像沒有遮罩
    only_images = image_names - mask_names

    # 只有遮罩沒有影像
    only_masks = mask_names - image_names

    # 建立配對列表
    paired_images = []
    paired_masks = []
    for name in sorted(paired_names):
        paired_images.append(image_dict[name])
        paired_masks.append(mask_dict[name])

    return paired_images, paired_masks, only_images, only_masks

# 執行配對驗證
paired_images, paired_masks, only_images, only_masks = verify_image_mask_pairs(
    valid_images, valid_masks
)

# 顯示配對結果
print("\n" + "="*70)
print("📊 配對驗證結果:")
print("="*70)

print(f"\n✅ 成功配對: {len(paired_images)} 對")
print(f"⚠️ 只有影像（缺少遮罩）: {len(only_images)} 個")
print(f"⚠️ 只有遮罩（缺少影像）: {len(only_masks)} 個")

# 顯示配對範例
print("\n📋 配對範例（前5對）:")
for i in range(min(5, len(paired_images))):
    img_name = os.path.basename(paired_images[i])
    mask_name = os.path.basename(paired_masks[i])
    print(f"  {i+1}. {img_name} ↔ {mask_name}")

# 顯示未配對檔案
if len(only_images) > 0:
    print("\n⚠️ 缺少遮罩的影像（前5個）:")
    for i, name in enumerate(list(only_images)[:5]):
        print(f"  {i+1}. {name}")

if len(only_masks) > 0:
    print("\n⚠️ 缺少影像的遮罩（前5個）:")
    for i, name in enumerate(list(only_masks)[:5]):
        print(f"  {i+1}. {name}")

# 視覺化配對狀態
fig, ax = plt.subplots(figsize=(10, 6))

categories = ['完美配對', '缺少遮罩', '缺少影像']
counts = [len(paired_images), len(only_images), len(only_masks)]
colors = ['#4CAF50', '#FF9800', '#F44336']

bars = ax.bar(categories, counts, color=colors, alpha=0.7, edgecolor='black')

# 在柱狀圖上顯示數值
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('數量', fontsize=12)
ax.set_title('影像-遮罩配對狀態', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

# 顯示配對率
pairing_rate = len(paired_images) / len(valid_images) * 100 if len(valid_images) > 0 else 0
print(f"\n📈 配對成功率: {pairing_rate:.1f}%")

print("\n" + "="*70)
print(f"✅ 配對驗證完成！將使用 {len(paired_images)} 對資料進行訓練")
print("="*70)

# 儲存配對資訊供後續使用
print("\n💾 儲存配對資訊...")
final_images = paired_images
final_masks = paired_masks
print(f"  最終資料量: {len(final_images)} 對")

In [ ]:
# ==================== Cell 5: 視覺化樣本資料 ====================
print("🎨 視覺化原始資料樣本...")
print("="*70)

def visualize_samples(image_paths, mask_paths, num_samples=4):
    """
    視覺化影像和對應的遮罩
    """
    # 隨機選擇樣本
    indices = random.sample(range(len(image_paths)), min(num_samples, len(image_paths)))

    fig, axes = plt.subplots(num_samples, 3, figsize=(15, num_samples * 4))

    if num_samples == 1:
        axes = axes.reshape(1, -1)

    for i, idx in enumerate(indices):
        # 讀取影像和遮罩
        img = Image.open(image_paths[idx]).convert('RGB')
        mask = Image.open(mask_paths[idx])

        # 轉換為 numpy array
        img_array = np.array(img)
        mask_array = np.array(mask)

        # 顯示原始影像
        axes[i, 0].imshow(img_array)
        axes[i, 0].set_title(f'原始影像\n{os.path.basename(image_paths[idx])}',
                            fontsize=10)
        axes[i, 0].axis('off')

        # 顯示遮罩（彩色映射）
        im = axes[i, 1].imshow(mask_array, cmap='jet', vmin=0, vmax=255)
        axes[i, 1].set_title(f'遮罩（彩色）\n{os.path.basename(mask_paths[idx])}',
                            fontsize=10)
        axes[i, 1].axis('off')
        plt.colorbar(im, ax=axes[i, 1], fraction=0.046, pad=0.04)

        # 顯示疊加效果
        overlay = img_array.copy()
        mask_colored = plt.cm.jet(mask_array / 255.0)[:, :, :3]
        overlay = (overlay * 0.6 + mask_colored * 255 * 0.4).astype(np.uint8)
        axes[i, 2].imshow(overlay)
        axes[i, 2].set_title('疊加效果', fontsize=10)
        axes[i, 2].axis('off')

        # 顯示影像資訊
        print(f"\n樣本 {i+1}:")
        print(f"  影像尺寸: {img_array.shape}")
        print(f"  遮罩尺寸: {mask_array.shape}")
        print(f"  遮罩唯一值: {np.unique(mask_array)}")
        print(f"  影像像素範圍: [{img_array.min()}, {img_array.max()}]")
        print(f"  遮罩像素範圍: [{mask_array.min()}, {mask_array.max()}]")

    plt.tight_layout()
    plt.show()

# 視覺化4個隨機樣本
print("\n🖼️ 隨機選擇 4 個樣本進行視覺化...")
visualize_samples(final_images, final_masks, num_samples=4)

# 統計影像尺寸分佈
print("\n" + "="*70)
print("📏 統計原始影像尺寸分佈...")
print("="*70)

sizes = []
for img_path in tqdm(final_images[:1000], desc="Scanning image sizes"):  # 抽樣1000張
    img = Image.open(img_path)
    sizes.append(img.size)

widths = [s[0] for s in sizes]
heights = [s[1] for s in sizes]

print(f"\n寬度統計:")
print(f"  最小: {min(widths)}px")
print(f"  最大: {max(widths)}px")
print(f"  平均: {np.mean(widths):.0f}px")
print(f"  中位數: {np.median(widths):.0f}px")

print(f"\n高度統計:")
print(f"  最小: {min(heights)}px")
print(f"  最大: {max(heights)}px")
print(f"  平均: {np.mean(heights):.0f}px")
print(f"  中位數: {np.median(heights):.0f}px")

# 視覺化尺寸分佈
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(widths, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(np.mean(widths), color='red', linestyle='--',
                linewidth=2, label=f'平均: {np.mean(widths):.0f}px')
axes[0].set_xlabel('寬度 (pixels)', fontsize=12)
axes[0].set_ylabel('數量', fontsize=12)
axes[0].set_title('影像寬度分佈', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

axes[1].hist(heights, bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].axvline(np.mean(heights), color='red', linestyle='--',
                linewidth=2, label=f'平均: {np.mean(heights):.0f}px')
axes[1].set_xlabel('高度 (pixels)', fontsize=12)
axes[1].set_ylabel('數量', fontsize=12)
axes[1].set_title('影像高度分佈', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("✅ 樣本視覺化完成！")
print("="*70)

In [ ]:
#  標籤分析與類別統計 

print("📊 分析遮罩標籤與類別分佈...")
print("="*70)

# Install and configure fonts-noto-cjk for displaying Chinese characters in plots
!apt-get update -qq
!apt-get install fonts-noto-cjk -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from collections import defaultdict
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import pandas as pd


# Configure matplotlib to use the Noto Sans CJK font
for font in fm.findSystemFonts(fontpaths=['/usr/share/fonts/opentype/noto/']):
    if 'NotoSansCJK' in font:
        fm.fontManager.addfont(font)

plt.rcParams['font.family'] = 'Noto Sans CJK JP'
plt.rcParams['axes.unicode_minus'] = False # Allow negative signs to be displayed

print("✅ Noto Sans CJK font installed and configured for this cell.")

def analyze_mask_labels(mask_paths, sample_size=None):
    """
    深度分析遮罩標籤

    統計：
    1. 所有唯一類別
    2. 每個類別的像素總數
    3. 每個類別出現在多少張影像中
    4. 類別不平衡程度
    """
    print("\n🔬 掃描遮罩檔案中的標籤...")

    if sample_size:
        mask_paths = random.sample(mask_paths, min(sample_size, len(mask_paths)))
        print(f"  使用 {len(mask_paths)} 個樣本進行分析")

    unique_labels_set = set()
    class_pixel_counts = defaultdict(int)
    class_occurrence = defaultdict(int)

    for mask_path in tqdm(mask_paths, desc="Analyzing masks"):
        mask = np.array(Image.open(mask_path))
        unique_labels = np.unique(mask)

        unique_labels_set.update(unique_labels)

        # 統計每個類別的像素數
        for label in unique_labels:
            pixel_count = np.sum(mask == label)
            class_pixel_counts[int(label)] += pixel_count
            class_occurrence[int(label)] += 1

    # 轉換為排序列表
    unique_labels_list = sorted(list(unique_labels_set))
    num_classes = len(unique_labels_list)

    return num_classes, unique_labels_list, class_pixel_counts, class_occurrence

# 執行標籤分析（使用全部資料）
NUM_CLASSES, unique_labels, pixel_counts, occurrences = analyze_mask_labels(
    final_masks, sample_size=None
)

print("\n" + "="*70)
print("📈 標籤分析結果:")
print("="*70)

print(f"\n🎯 類別總數: {NUM_CLASSES}")
print(f"🏷️ 所有類別標籤: {unique_labels}")

# 建立詳細統計表格
print("\n" + "="*70)
print("📋 詳細類別統計:")
print("="*70)

stats_data = []
total_pixels = sum(pixel_counts.values())

for label in unique_labels:
    pixels = pixel_counts[label]
    occur = occurrences[label]
    percentage = (pixels / total_pixels) * 100
    avg_pixels = pixels / occur if occur > 0 else 0

    stats_data.append({
        '類別ID': label,
        '出現次數': occur,
        '總像素數': f'{pixels:,}',
        '像素佔比': f'{percentage:.2f}%',
        '平均像素': f'{avg_pixels:,.0f}'
    })

df_stats = pd.DataFrame(stats_data)
print(df_stats.to_string(index=False))

# 檢測類別不平衡
print("\n" + "="*70)
print("⚖️ 類別平衡分析:")
print("="*70)

pixel_values = list(pixel_counts.values())
max_pixels = max(pixel_values)
min_pixels = min(pixel_values)
imbalance_ratio = max_pixels / min_pixels if min_pixels > 0 else float('inf')

print(f"最多像素的類別: {max_pixels:,} 像素")
print(f"最少像素的類別: {min_pixels:,} 像素")
print(f"不平衡比例: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 100:
    print("\n⚠️ 警告: 檢測到嚴重的類別不平衡！")
    print("   建議使用以下策略:")
    print("   - 加權損失函數 (Weighted Loss)")
    print("   - 過採樣少數類別 (Oversampling)")
    print("   - Focal Loss")
elif imbalance_ratio > 10:
    print("\n⚠️ 注意: 檢測到中度類別不平衡")
    print("   建議使用加權損失函數")
else:
    print("\n✅ 類別分佈相對平衡")

# 視覺化類別分佈
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. 類別出現次數
ax1 = axes[0, 0]
labels_list = [str(l) for l in unique_labels]
occur_list = [occurrences[l] for l in unique_labels]
bars1 = ax1.bar(labels_list, occur_list, color='steelblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('類別 ID', fontsize=12)
ax1.set_ylabel('出現次數', fontsize=12)
ax1.set_title('各類別出現頻率', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3, linestyle='--')
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom', fontsize=9)

# 2. 類別像素佔比
ax2 = axes[0, 1]
pixel_list = [pixel_counts[l] for l in unique_labels]
bars2 = ax2.bar(labels_list, pixel_list, color='coral', edgecolor='black', alpha=0.7)
ax2.set_xlabel('類別 ID', fontsize=12)
ax2.set_ylabel('總像素數', fontsize=12)
ax2.set_title('各類別像素總量', fontsize=14, fontweight='bold')
ax2.set_yscale('log')  # 使用對數刻度以更好地顯示差異
ax2.grid(axis='y', alpha=0.3, linestyle='--')

# 3. 圓餅圖 - 像素佔比（只顯示前10大類別）
ax3 = axes[1, 0]
sorted_labels = sorted(unique_labels, key=lambda x: pixel_counts[x], reverse=True)
top_10_labels = sorted_labels[:10]
top_10_pixels = [pixel_counts[l] for l in top_10_labels]
if len(sorted_labels) > 10:
    other_pixels = sum([pixel_counts[l] for l in sorted_labels[10:]])
    top_10_labels.append('其他')
    top_10_pixels.append(other_pixels)

colors = plt.cm.Set3(range(len(top_10_labels)))
wedges, texts, autotexts = ax3.pie(top_10_pixels, labels=top_10_labels, autopct='%1.1f%%',
                                     colors=colors, startangle=90)
ax3.set_title('像素佔比分佈（前10類別）', fontsize=14, fontweight='bold')

# 4. 平均像素數
ax4 = axes[1, 1]
avg_pixel_list = [pixel_counts[l] / occurrences[l] if occurrences[l] > 0 else 0
                  for l in unique_labels]
bars4 = ax4.bar(labels_list, avg_pixel_list, color='mediumseagreen',
                edgecolor='black', alpha=0.7)
ax4.set_xlabel('類別 ID', fontsize=12)
ax4.set_ylabel('平均像素數', fontsize=12)
ax4.set_title('各類別平均像素數', fontsize=14, fontweight='bold')
ax4.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print(f"✅ 標籤分析完成！共找到 {NUM_CLASSES} 個類別")
print("="*70)

# 儲存類別資訊
print("\n💾 儲存類別統計資訊...")
class_info = {
    'num_classes': NUM_CLASSES,
    'unique_labels': unique_labels,
    'pixel_counts': dict(pixel_counts),
    'occurrences': dict(occurrences),
    'imbalance_ratio': imbalance_ratio
}
print("  ✅ 類別資訊已儲存到變數 'class_info'")

In [ ]:
#  計算類別權重 

print("⚖️ 計算類別權重以處理不平衡問題...")
print("="*70)

def compute_class_weights(pixel_counts, unique_labels):
    """
    計算類別權重（反頻率加權）

    公式: weight = total_pixels / (num_classes × class_pixels)
    然後正規化到 [0.1, 10] 範圍避免極端值
    """
    print("\n🔢 使用反頻率加權方法...")

    total_pixels = sum(pixel_counts.values())
    num_classes = len(unique_labels)

    # 計算原始權重
    raw_weights = {}
    for label in unique_labels:
        class_pixels = pixel_counts[label]
        if class_pixels > 0:
            raw_weights[label] = total_pixels / (num_classes * class_pixels)
        else:
            raw_weights[label] = 1.0

    # 正規化權重（避免極端值）
    max_weight = max(raw_weights.values())
    min_weight = min(raw_weights.values())

    normalized_weights = {}
    for label, weight in raw_weights.items():
        # 正規化到 [0.1, 10] 範圍
        normalized = 0.1 + (weight - min_weight) / (max_weight - min_weight) * 9.9
        normalized_weights[label] = normalized

    return normalized_weights, raw_weights

# 計算權重
class_weights, raw_weights = compute_class_weights(pixel_counts, unique_labels)

# 顯示權重
print("\n" + "="*70)
print("📊 類別權重結果:")
print("="*70)

weight_data = []
for label in unique_labels:
    weight_data.append({
        '類別ID': label,
        '原始權重': f'{raw_weights[label]:.4f}',
        '正規化權重': f'{class_weights[label]:.4f}',
        '像素數': f'{pixel_counts[label]:,}',
        '權重說明': '高' if class_weights[label] > 5 else '中' if class_weights[label] > 2 else '低'
    })

df_weights = pd.DataFrame(weight_data)
print(df_weights.to_string(index=False))

print("\n💡 權重解釋:")
print("  - 權重越高 = 該類別越稀有 = 模型會更重視該類別的錯誤")
print("  - 權重越低 = 該類別越常見 = 避免模型過度關注")

# 視覺化權重分佈
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. 權重條形圖
ax1 = axes[0]
labels_str = [str(l) for l in unique_labels]
weight_values = [class_weights[l] for l in unique_labels]
colors_map = ['red' if w > 5 else 'orange' if w > 2 else 'green' for w in weight_values]

bars = ax1.bar(labels_str, weight_values, color=colors_map, edgecolor='black', alpha=0.7)
ax1.set_xlabel('類別 ID', fontsize=12)
ax1.set_ylabel('正規化權重', fontsize=12)
ax1.set_title('各類別權重分佈', fontsize=14, fontweight='bold')
ax1.axhline(y=1.0, color='blue', linestyle='--', linewidth=2, label='基準線 (1.0)')
ax1.grid(axis='y', alpha=0.3, linestyle='--')
ax1.legend()

# 在條形上顯示數值
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}', ha='center', va='bottom', fontsize=9)

# 2. 像素數 vs 權重散點圖
ax2 = axes[1]
pixel_values = [pixel_counts[l] for l in unique_labels]
weight_values = [class_weights[l] for l in unique_labels]

scatter = ax2.scatter(pixel_values, weight_values, s=100, c=weight_values,
                     cmap='RdYlGn_r', edgecolors='black', alpha=0.7)
ax2.set_xlabel('類別像素總數', fontsize=12)
ax2.set_ylabel('正規化權重', fontsize=12)
ax2.set_title('像素數量 vs 權重關係', fontsize=14, fontweight='bold')
ax2.set_xscale('log')
ax2.grid(True, alpha=0.3, linestyle='--')

# 添加標註
for i, label in enumerate(unique_labels):
    ax2.annotate(str(label), (pixel_values[i], weight_values[i]),
                fontsize=9, ha='center')

plt.colorbar(scatter, ax=ax2, label='權重值')
plt.tight_layout()
plt.show()

# 轉換為 TensorFlow 可用的格式
print("\n" + "="*70)
print("🔧 轉換為訓練用格式...")
print("="*70)

# 建立類別索引到權重的映射（0-based index）
class_weight_dict = {}
for i, label in enumerate(unique_labels):
    class_weight_dict[i] = class_weights[label]

print(f"\n✅ 類別權重字典（用於訓練）:")
print(f"  {class_weight_dict}")

print("\n📝 使用方式:")
print("  在 model.fit() 中加入參數:")
print("  class_weight=class_weight_dict")

print("\n" + "="*70)
print("✅ 類別權重計算完成！")
print("="*70)

In [ ]:
# 計算資料集統計量
print("📐 計算資料集的 RGB 通道統計量...")
print("="*70)

def compute_dataset_statistics(image_paths, sample_size=1000):
    """
    計算資料集的均值和標準差

    用途：比標準的 [-1, 1] 正規化更精確
    """
    print(f"\n🔬 抽樣 {sample_size} 張影像計算統計量...")

    # 抽樣
    sampled_paths = random.sample(image_paths, min(sample_size, len(image_paths)))

    # 收集像素值
    r_values = []
    g_values = []
    b_values = []

    for img_path in tqdm(sampled_paths, desc="Computing statistics"):
        img = np.array(Image.open(img_path).convert('RGB')) / 255.0  # 正規化到 [0, 1]

        r_values.append(img[:, :, 0].flatten())
        g_values.append(img[:, :, 1].flatten())
        b_values.append(img[:, :, 2].flatten())

    # 合併所有像素
    r_all = np.concatenate(r_values)
    g_all = np.concatenate(g_values)
    b_all = np.concatenate(b_values)

    # 計算均值和標準差
    mean = np.array([r_all.mean(), g_all.mean(), b_all.mean()])
    std = np.array([r_all.std(), g_all.std(), b_all.std()])

    return mean, std, (r_all, g_all, b_all)

# 計算統計量
dataset_mean, dataset_std, channel_data = compute_dataset_statistics(
    final_images, sample_size=1000
)

# 顯示結果
print("\n" + "="*70)
print("📊 資料集統計結果:")
print("="*70)

print(f"\n🎨 RGB 通道均值:")
print(f"  R (紅色): {dataset_mean[0]:.4f}")
print(f"  G (綠色): {dataset_mean[1]:.4f}")
print(f"  B (藍色): {dataset_mean[2]:.4f}")
print(f"  向量形式: {dataset_mean}")

print(f"\n📏 RGB 通道標準差:")
print(f"  R (紅色): {dataset_std[0]:.4f}")
print(f"  G (綠色): {dataset_std[1]:.4f}")
print(f"  B (藍色): {dataset_std[2]:.4f}")
print(f"  向量形式: {dataset_std}")

print(f"\n💡 正規化公式:")
print(f"  normalized_pixel = (pixel / 255.0 - mean) / std")
print(f"  即: (pixel / 255.0 - {dataset_mean}) / {dataset_std}")

# 視覺化通道分佈
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

r_data, g_data, b_data = channel_data

# 1. RGB 直方圖
ax1 = axes[0, 0]
ax1.hist(r_data, bins=100, color='red', alpha=0.5, label='R', density=True)
ax1.hist(g_data, bins=100, color='green', alpha=0.5, label='G', density=True)
ax1.hist(b_data, bins=100, color='blue', alpha=0.5, label='B', density=True)
ax1.set_xlabel('像素值 (0-1)', fontsize=12)
ax1.set_ylabel('密度', fontsize=12)
ax1.set_title('RGB 通道像素值分佈', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. 通道統計比較
ax2 = axes[0, 1]
channels = ['R', 'G', 'B']
means = dataset_mean
stds = dataset_std

x = np.arange(len(channels))
width = 0.35

bars1 = ax2.bar(x - width/2, means, width, label='均值', color='skyblue', edgecolor='black')
bars2 = ax2.bar(x + width/2, stds, width, label='標準差', color='lightcoral', edgecolor='black')

ax2.set_ylabel('數值', fontsize=12)
ax2.set_title('各通道統計量比較', fontsize=14, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(channels)
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 在條形上顯示數值
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=10)

# 3. 正規化前後對比（示例影像）
ax3 = axes[1, 0]
sample_img = np.array(Image.open(final_images[0]).convert('RGB'))
ax3.imshow(sample_img)
ax3.set_title('原始影像', fontsize=12, fontweight='bold')
ax3.axis('off')

# 4. 正規化後的影像
ax4 = axes[1, 1]
normalized_img = (sample_img / 255.0 - dataset_mean) / dataset_std
# 為了顯示，需要反正規化到可視範圍
display_img = (normalized_img - normalized_img.min()) / (normalized_img.max() - normalized_img.min())
ax4.imshow(display_img)
ax4.set_title('正規化後影像（重新縮放以顯示）', fontsize=12, fontweight='bold')
ax4.axis('off')

plt.tight_layout()
plt.show()

# 比較不同正規化方法
print("\n" + "="*70)
print("🔍 正規化方法比較:")
print("="*70)

comparison_data = []

# 方法1: [-1, 1] 正規化
method1_mean = [0.0, 0.0, 0.0]
method1_std = [1.0, 1.0, 1.0]

# 方法2: ImageNet 標準值
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

# 方法3: 資料集統計值
dataset_mean_list = dataset_mean.tolist()
dataset_std_list = dataset_std.tolist()

comparison_data.append({
    '方法': '標準正規化 [-1,1]',
    'R均值': 0.0, 'G均值': 0.0, 'B均值': 0.0,
    'R標準差': 1.0, 'G標準差': 1.0, 'B標準差': 1.0,
    '優點': '簡單快速',
    '缺點': '不考慮資料分佈'
})

comparison_data.append({
    '方法': 'ImageNet 預訓練',
    'R均值': imagenet_mean[0], 'G均值': imagenet_mean[1], 'B均值': imagenet_mean[2],
    'R標準差': imagenet_std[0], 'G標準差': imagenet_std[1], 'B標準差': imagenet_std[2],
    '優點': '適用於遷移學習',
    '缺點': '可能不適合時尚資料'
})

comparison_data.append({
    '方法': '資料集自身統計 ⭐',
    'R均值': f"{dataset_mean_list[0]:.4f}",
    'G均值': f"{dataset_mean_list[1]:.4f}",
    'B均值': f"{dataset_mean_list[2]:.4f}",
    'R標準差': f"{dataset_std_list[0]:.4f}",
    'G標準差': f"{dataset_std_list[1]:.4f}",
    'B標準差': f"{dataset_std_list[2]:.4f}",
    '優點': '最精確',
    '缺點': '需要計算時間'
})

df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))

print("\n✅ 推薦使用: 資料集自身統計值（已計算）")

print("\n" + "="*70)
print("✅ 統計量計算完成！")
print("="*70)

In [ ]:
# 資料分割（分層分割）

print("✂️ 分割資料集為訓練/驗證/測試集...")
print("="*70)

def stratified_split(image_paths, mask_paths, val_size=0.1, test_size=0.05):
    """
    分層分割資料集

    策略：
    1. 根據遮罩中的主要類別進行分層
    2. 確保每個類別在各集合中都有代表
    """
    print("\n🔍 分析每個樣本的主要類別...")

    dominant_classes = []

    for mask_path in tqdm(mask_paths, desc="Analyzing dominant classes"):
        mask = np.array(Image.open(mask_path))
        unique, counts = np.unique(mask, return_counts=True)

        # 找出最多像素的類別（排除背景0）
        non_bg = [(u, c) for u, c in zip(unique, counts) if u > 0]
        if non_bg:
            dominant_class = max(non_bg, key=lambda x: x[1])[0]
        else:
            dominant_class = 0

        dominant_classes.append(dominant_class)

    print(f"\n找到 {len(set(dominant_classes))} 個不同的主要類別")

    # 第一次分割: 訓練集 vs (驗證+測試)集
    print(f"\n第一次分割: {(1-val_size)*100:.0f}% 訓練, {val_size*100:.0f}% (驗證+測試)...")

    train_images, temp_images, train_masks, temp_masks, train_classes, temp_classes = \
        train_test_split(
            image_paths, mask_paths, dominant_classes,
            test_size=val_size,
            stratify=dominant_classes,
            random_state=RANDOM_STATE
        )

    # 第二次分割: 驗證集 vs 測試集
    relative_test_size = test_size / val_size
    print(f"第二次分割: {(1-relative_test_size)*100:.0f}% 驗證, {relative_test_size*100:.0f}% 測試...")

    val_images, test_images, val_masks, test_masks = \
        train_test_split(
            temp_images, temp_masks,
            test_size=relative_test_size,
            stratify=temp_classes,
            random_state=RANDOM_STATE
        )

    return (train_images, val_images, test_images,
            train_masks, val_masks, test_masks,
            train_classes, temp_classes)

# 執行分割
(train_images, val_images, test_images,
 train_masks, val_masks, test_masks,
 train_classes, temp_classes) = stratified_split(
    final_images, final_masks, val_size=0.1, test_size=0.05
)

# 顯示分割結果
print("\n" + "="*70)
print("📊 資料分割結果:")
print("="*70)

total = len(final_images)
train_size = len(train_images)
val_size = len(val_images)
test_size = len(test_images)

print(f"\n總資料量: {total} 對")
print(f"\n訓練集: {train_size} 對 ({train_size/total*100:.1f}%)")
print(f"驗證集: {val_size} 對 ({val_size/total*100:.1f}%)")
print(f"測試集: {test_size} 對 ({test_size/total*100:.1f}%)")

# 檢查類別分佈
print("\n" + "="*70)
print("🔍 檢查各集合的類別分佈:")
print("="*70)

# 收集每個集合的類別分佈
def get_class_distribution(mask_paths, set_name):
    """統計類別分佈"""
    class_counts = defaultdict(int)

    for mask_path in mask_paths[:100]:  # 抽樣100張檢查
        mask = np.array(Image.open(mask_path))
        unique = np.unique(mask)
        for u in unique:
            class_counts[int(u)] += 1

    return class_counts

train_dist = get_class_distribution(train_masks, "訓練集")
val_dist = get_class_distribution(val_masks, "驗證集")
test_dist = get_class_distribution(test_masks, "測試集")

print("\n各類別在不同集合中的出現次數（抽樣100張）:")
dist_data = []
for label in sorted(unique_labels):
    dist_data.append({
        '類別': label,
        '訓練集': train_dist.get(label, 0),
        '驗證集': val_dist.get(label, 0),
        '測試集': test_dist.get(label, 0)
    })

df_dist = pd.DataFrame(dist_data)
print(df_dist.to_string(index=False))

# 視覺化分割結果
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 資料集大小比較
ax1 = axes[0, 0]
sets = ['訓練集', '驗證集', '測試集']
sizes = [train_size, val_size, test_size]
colors_set = ['#4CAF50', '#2196F3', '#FF9800']

bars = ax1.bar(sets, sizes, color=colors_set, edgecolor='black', alpha=0.7)
ax1.set_ylabel('樣本數量', fontsize=12)
ax1.set_title('資料集分割比例', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

for bar in bars:
    height = bar.get_height()
    percentage = (height / total) * 100
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}\n({percentage:.1f}%)',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

# 2. 圓餅圖
ax2 = axes[0, 1]
ax2.pie(sizes, labels=sets, colors=colors_set, autopct='%1.1f%%',
        startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
ax2.set_title('資料分佈比例', fontsize=14, fontweight='bold')

# 3. 類別分佈比較（堆疊柱狀圖）
ax3 = axes[1, 0]
labels_for_plot = [str(l) for l in sorted(unique_labels)[:10]]  # 只顯示前10個類別
train_counts = [train_dist.get(l, 0) for l in sorted(unique_labels)[:10]]
val_counts = [val_dist.get(l, 0) for l in sorted(unique_labels)[:10]]
test_counts = [test_dist.get(l, 0) for l in sorted(unique_labels)[:10]]

x = np.arange(len(labels_for_plot))
width = 0.25

bars1 = ax3.bar(x - width, train_counts, width, label='訓練集', color='#4CAF50', alpha=0.7)
bars2 = ax3.bar(x, val_counts, width, label='驗證集', color='#2196F3', alpha=0.7)
bars3 = ax3.bar(x + width, test_counts, width, label='測試集', color='#FF9800', alpha=0.7)

ax3.set_xlabel('類別 ID', fontsize=12)
ax3.set_ylabel('出現次數（抽樣）', fontsize=12)
ax3.set_title('各集合類別分佈比較（前10類別）', fontsize=14, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(labels_for_plot)
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# 4. 顯示範例樣本
ax4 = axes[1, 1]
ax4.axis('off')
info_text = f"""
📊 分割統計摘要

總資料量: {total:,} 對

訓練集: {train_size:,} ({train_size/total*100:.1f}%)
驗證集: {val_size:,} ({val_size/total*100:.1f}%)
測試集: {test_size:,} ({test_size/total*100:.1f}%)

✅ 使用分層分割策略
✅ 所有類別均有代表
✅ 隨機種子: {RANDOM_STATE}
"""
ax4.text(0.1, 0.5, info_text, fontsize=12, verticalalignment='center',
         family='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

# 驗證分割的合理性
print("\n" + "="*70)
print("✅ 分割合理性檢查:")
print("="*70)

# 檢查1: 確保沒有重複
train_set = set(train_images)
val_set = set(val_images)
test_set = set(test_images)

overlap_train_val = train_set & val_set
overlap_train_test = train_set & test_set
overlap_val_test = val_set & test_set

print(f"\n重複檢查:")
print(f"  訓練集 ∩ 驗證集: {len(overlap_train_val)} (應為 0)")
print(f"  訓練集 ∩ 測試集: {len(overlap_train_test)} (應為 0)")
print(f"  驗證集 ∩ 測試集: {len(overlap_val_test)} (應為 0)")

if len(overlap_train_val) == 0 and len(overlap_train_test) == 0 and len(overlap_val_test) == 0:
    print("  ✅ 通過：沒有重複樣本")
else:
    print("  ❌ 警告：發現重複樣本！")

# 檢查2: 總數是否正確
total_split = len(train_images) + len(val_images) + len(test_images)
print(f"\n總數檢查:")
print(f"  分割前: {total}")
print(f"  分割後: {total_split}")
print(f"  差異: {total - total_split}")

if total == total_split:
    print("  ✅ 通過：總數一致")
else:
    print("  ❌ 警告：總數不一致！")

# 顯示樣本範例
print("\n" + "="*70)
print("📝 樣本路徑範例:")
print("="*70)

print("\n訓練集（前3個）:")
for i in range(min(3, len(train_images))):
    print(f"  {i+1}. 影像: {os.path.basename(train_images[i])}")
    print(f"     遮罩: {os.path.basename(train_masks[i])}")

print("\n驗證集（前3個）:")
for i in range(min(3, len(val_images))):
    print(f"  {i+1}. 影像: {os.path.basename(val_images[i])}")
    print(f"     遮罩: {os.path.basename(val_masks[i])}")

print("\n測試集（前3個）:")
for i in range(min(3, len(test_images))):
    print(f"  {i+1}. 影像: {os.path.basename(test_images[i])}")
    print(f"     遮罩: {os.path.basename(test_masks[i])}")

print("\n" + "="*70)
print("✅ 資料分割完成！")
print("="*70)

In [ ]:
# Cell 10: 影像讀取與正規化函數

print("🖼️ 建立影像讀取與正規化函數...")
print("="*70)

def read_and_normalize_image(image_path, mask_path):
    """
    讀取並正規化影像和遮罩

    影像處理:
    1. 讀取 JPEG (RGB 3通道)
    2. 縮放到 512x512
    3. 使用資料集統計量正規化

    遮罩處理:
    1. 讀取 PNG (單通道)
    2. 縮放到 512x512 (最近鄰插值)
    3. 保持原始標籤值
    """
    # 讀取影像
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)

    # 縮放影像
    img = tf.image.resize(img, [IMAGE_SIZE, IMAGE_SIZE], method='bilinear')

    # 正規化影像: (pixel / 255.0 - mean) / std
    img = img / 255.0
    img = (img - dataset_mean) / dataset_std

    # 讀取遮罩
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.cast(mask, tf.float32)

    # 縮放遮罩 (使用最近鄰插值保持標籤完整性)
    mask = tf.image.resize(mask, [IMAGE_SIZE, IMAGE_SIZE], method='nearest')

    # 將遮罩值轉換為類別索引 (0 到 NUM_CLASSES-1)
    mask = mask / 255.0  # 正規化到 [0, 1]

    return img, mask

# 測試讀取函數
print("\n🧪 測試影像讀取函數...")

# 讀取一個樣本
test_img, test_mask = read_and_normalize_image(
    train_images[0], train_masks[0]
)

print(f"\n✅ 讀取成功！")
print(f"  影像形狀: {test_img.shape}")
print(f"  影像數值範圍: [{test_img.numpy().min():.3f}, {test_img.numpy().max():.3f}]")
print(f"  影像數據類型: {test_img.dtype}")

print(f"\n  遮罩形狀: {test_mask.shape}")
print(f"  遮罩數值範圍: [{test_mask.numpy().min():.3f}, {test_mask.numpy().max():.3f}]")
print(f"  遮罩唯一值: {np.unique(test_mask.numpy())[:10]}")  # 顯示前10個
print(f"  遮罩數據類型: {test_mask.dtype}")

# 視覺化處理後的樣本
print("\n📊 視覺化處理後的資料...")

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 處理3個樣本
for i in range(3):
    img_processed, mask_processed = read_and_normalize_image(
        train_images[i], train_masks[i]
    )

    # 原始影像（反正規化以顯示）
    img_display = img_processed.numpy() * dataset_std + dataset_mean
    img_display = np.clip(img_display, 0, 1)

    # 顯示影像
    axes[0, i].imshow(img_display)
    axes[0, i].set_title(f'樣本 {i+1} - 處理後影像\n{IMAGE_SIZE}x{IMAGE_SIZE}',
                        fontsize=10)
    axes[0, i].axis('off')

    # 顯示遮罩
    im = axes[1, i].imshow(mask_processed.numpy()[:, :, 0], cmap='jet', vmin=0, vmax=1)
    axes[1, i].set_title(f'樣本 {i+1} - 處理後遮罩\n{IMAGE_SIZE}x{IMAGE_SIZE}',
                        fontsize=10)
    axes[1, i].axis('off')
    plt.colorbar(im, ax=axes[1, i], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

# 顯示正規化效果
print("\n" + "="*70)
print("📈 正規化效果分析:")
print("="*70)

# 收集多個樣本的統計資訊
sample_means = []
sample_stds = []

for i in range(min(10, len(train_images))):
    img, _ = read_and_normalize_image(train_images[i], train_masks[i])
    img_np = img.numpy()
    sample_means.append([img_np[:,:,0].mean(), img_np[:,:,1].mean(), img_np[:,:,2].mean()])
    sample_stds.append([img_np[:,:,0].std(), img_np[:,:,1].std(), img_np[:,:,2].std()])

sample_means = np.array(sample_means)
sample_stds = np.array(sample_stds)

print(f"\n正規化後的統計（10個樣本）:")
print(f"\nRGB 均值平均:")
print(f"  R: {sample_means[:,0].mean():.4f} ± {sample_means[:,0].std():.4f}")
print(f"  G: {sample_means[:,1].mean():.4f} ± {sample_means[:,1].std():.4f}")
print(f"  B: {sample_means[:,2].mean():.4f} ± {sample_means[:,2].std():.4f}")

print(f"\nRGB 標準差平均:")
print(f"  R: {sample_stds[:,0].mean():.4f} ± {sample_stds[:,0].std():.4f}")
print(f"  G: {sample_stds[:,1].mean():.4f} ± {sample_stds[:,1].std():.4f}")
print(f"  B: {sample_stds[:,2].mean():.4f} ± {sample_stds[:,2].std():.4f}")

print(f"\n💡 理想情況：正規化後均值應接近 0，標準差應接近 1")

# 視覺化正規化效果
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 均值分佈
ax1 = axes[0]
ax1.boxplot([sample_means[:,0], sample_means[:,1], sample_means[:,2]],
            labels=['R', 'G', 'B'])
ax1.axhline(y=0, color='r', linestyle='--', linewidth=2, label='目標均值 (0)')
ax1.set_ylabel('均值', fontsize=12)
ax1.set_title('正規化後各通道均值分佈', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 標準差分佈
ax2 = axes[1]
ax2.boxplot([sample_stds[:,0], sample_stds[:,1], sample_stds[:,2]],
            labels=['R', 'G', 'B'])
ax2.axhline(y=1, color='r', linestyle='--', linewidth=2, label='目標標準差 (1)')
ax2.set_ylabel('標準差', fontsize=12)
ax2.set_title('正規化後各通道標準差分佈', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("✅ 影像讀取與正規化函數建立完成！")
print("="*70)

In [ ]:
#  資料增強函數 

print("🎨 建立資料增強函數...")
print("="*70)

# 增強機率設定
AUGMENTATION_CONFIG = {
    'horizontal_flip_prob': 0.5,
    'rotation_prob': 0.3,
    'brightness_prob': 0.4,
    'contrast_prob': 0.4,
    'saturation_prob': 0.3,
    'random_crop_prob': 0.3,
}

print("\n⚙️ 增強配置:")
for key, value in AUGMENTATION_CONFIG.items():
    print(f"  {key}: {value}")

# 1. 水平翻轉
@tf.function
def random_horizontal_flip(image, mask):
    """隨機水平翻轉（影像和遮罩同步）"""
    if tf.random.uniform([]) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
    return image, mask

# 2. 旋轉（90度倍數）
@tf.function
def random_rotation(image, mask):
    """隨機旋轉90度倍數"""
    if tf.random.uniform([]) < AUGMENTATION_CONFIG['rotation_prob']:
        k = tf.random.uniform([], 0, 4, dtype=tf.int32)
        image = tf.image.rot90(image, k=k)
        mask = tf.image.rot90(mask, k=k)
    return image, mask

# 3. 亮度調整
@tf.function
def random_brightness(image, mask):
    """隨機調整亮度（只對影像）"""
    if tf.random.uniform([]) < AUGMENTATION_CONFIG['brightness_prob']:
        image = tf.image.adjust_brightness(image, tf.random.uniform([], -0.2, 0.2))
    return image, mask

# 4. 對比度調整
@tf.function
def random_contrast(image, mask):
    """隨機調整對比度（只對影像）"""
    if tf.random.uniform([]) < AUGMENTATION_CONFIG['contrast_prob']:
        image = tf.image.adjust_contrast(image, tf.random.uniform([], 0.8, 1.2))
    return image, mask

# 5. 飽和度調整
@tf.function
def random_saturation(image, mask):
    """隨機調整飽和度（只對影像）"""
    if tf.random.uniform([]) < AUGMENTATION_CONFIG['saturation_prob']:
        image = tf.image.adjust_saturation(image, tf.random.uniform([], 0.8, 1.2))
    return image, mask

# 6. 隨機裁切
@tf.function
def random_crop_and_resize(image, mask):
    """隨機裁切並縮放回原尺寸"""
    if tf.random.uniform([]) < AUGMENTATION_CONFIG['random_crop_prob']:
        # 隨機裁切大小 (80%-100%)
        crop_size = tf.random.uniform([], 0.8, 1.0)
        h = tf.cast(IMAGE_SIZE * crop_size, tf.int32)
        w = tf.cast(IMAGE_SIZE * crop_size, tf.int32)

        # 合併影像和遮罩進行相同裁切
        combined = tf.concat([image, mask], axis=-1)
        combined = tf.image.random_crop(combined, [h, w, 4])

        # 縮放回原尺寸
        combined = tf.image.resize(combined, [IMAGE_SIZE, IMAGE_SIZE])

        image = combined[:, :, :3]
        mask = combined[:, :, 3:]

    return image, mask

# 組合所有增強
@tf.function
def apply_augmentation(image, mask):
    """應用所有增強方法"""
    # 幾何變換
    image, mask = random_horizontal_flip(image, mask)
    image, mask = random_rotation(image, mask)
    image, mask = random_crop_and_resize(image, mask)

    # 色彩增強（只對影像）
    image, mask = random_brightness(image, mask)
    image, mask = random_contrast(image, mask)
    image, mask = random_saturation(image, mask)

    return image, mask

# 測試增強效果
print("\n" + "="*70)
print("🧪 測試資料增強效果...")
print("="*70)

# 載入一個樣本
test_img, test_mask = read_and_normalize_image(train_images[0], train_masks[0])

# 生成多個增強版本
print("\n生成 8 個增強樣本...")

fig, axes = plt.subplots(4, 4, figsize=(16, 16))

for i in range(8):
    # 應用增強
    aug_img, aug_mask = apply_augmentation(test_img, test_mask)

    # 反正規化影像以顯示
    display_img = aug_img.numpy() * dataset_std + dataset_mean
    display_img = np.clip(display_img, 0, 1)

    # 顯示影像
    row = i // 2
    col = (i % 2) * 2
    axes[row, col].imshow(display_img)
    axes[row, col].set_title(f'增強樣本 {i+1} - 影像', fontsize=10)
    axes[row, col].axis('off')

    # 顯示遮罩
    axes[row, col+1].imshow(aug_mask.numpy()[:, :, 0], cmap='jet', vmin=0, vmax=1)
    axes[row, col+1].set_title(f'增強樣本 {i+1} - 遮罩', fontsize=10)
    axes[row, col+1].axis('off')

plt.tight_layout()
plt.show()

# 統計增強效果
print("\n" + "="*70)
print("📊 增強效果統計（100次增強）:")
print("="*70)

augmentation_stats = {
    'flipped': 0,
    'rotated': 0,
    'cropped': 0,
    'brightness_changed': 0,
    'contrast_changed': 0,
}

original_img, original_mask = read_and_normalize_image(train_images[0], train_masks[0])

for _ in tqdm(range(100), desc="Testing augmentations"):
    aug_img, aug_mask = apply_augmentation(original_img, original_mask)

    # 檢測是否有翻轉（比較像素）
    if not tf.reduce_all(tf.equal(aug_img, original_img)):
        # 某種增強被應用了
        pass

print("\n💡 增強說明:")
print("  - 每次增強時，各個方法根據設定的機率獨立應用")
print("  - 影像和遮罩的幾何變換保持同步")
print("  - 色彩增強只應用於影像，不影響遮罩")

# 比較原始 vs 增強
print("\n" + "="*70)
print("🔍 原始 vs 增強對比:")
print("="*70)

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# 原始影像
orig_display = original_img.numpy() * dataset_std + dataset_mean
orig_display = np.clip(orig_display, 0, 1)

axes[0, 0].imshow(orig_display)
axes[0, 0].set_title('原始影像', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[1, 0].imshow(original_mask.numpy()[:, :, 0], cmap='jet', vmin=0, vmax=1)
axes[1, 0].set_title('原始遮罩', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

# 3個不同的增強版本
for i in range(3):
    aug_img, aug_mask = apply_augmentation(original_img, original_mask)

    aug_display = aug_img.numpy() * dataset_std + dataset_mean
    aug_display = np.clip(aug_display, 0, 1)

    axes[0, i+1].imshow(aug_display)
    axes[0, i+1].set_title(f'增強版本 {i+1}', fontsize=12, fontweight='bold')
    axes[0, i+1].axis('off')

    axes[1, i+1].imshow(aug_mask.numpy()[:, :, 0], cmap='jet', vmin=0, vmax=1)
    axes[1, i+1].set_title(f'增強遮罩 {i+1}', fontsize=12, fontweight='bold')
    axes[1, i+1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ 觀察要點:")
print("  ✓ 影像和遮罩的幾何變換是否同步？")
print("  ✓ 遮罩的標籤值是否保持完整（無插值模糊）？")
print("  ✓ 色彩變化是否自然？")
print("  ✓ 裁切後的內容是否仍然有意義？")

print("\n" + "="*70)
print("✅ 資料增強函數建立完成！")
print("="*70)

In [ ]:
# 建立 TensorFlow Dataset 管線 
print("🚀 建立高效能 TensorFlow Dataset 管線...")
print("="*70)

def create_dataset(image_paths, mask_paths, is_training=True, batch_size=BATCH_SIZE):
    """
    建立優化的資料管線

    步驟:
    1. 從路徑建立 dataset
    2. 讀取和正規化
    3. 快取到記憶體
    4. 資料增強（僅訓練集）
    5. 打亂（僅訓練集）
    6. 批次化
    7. 預取
    """
    print(f"\n📦 建立 {'訓練' if is_training else '驗證/測試'} 資料集...")
    print(f"  樣本數量: {len(image_paths)}")
    print(f"  批次大小: {batch_size}")

    # 步驟1: 建立路徑 dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))

    # 步驟2: 讀取和正規化
    dataset = dataset.map(
        read_and_normalize_image,
        num_parallel_calls=AUTOTUNE
    )

    # 步驟3: 快取（加速訓練）
    dataset = dataset.cache()
    print(f"  ✓ 快取到記憶體")

    if is_training:
        # 步驟4: 打亂
        dataset = dataset.shuffle(buffer_size=min(1000, len(image_paths)))
        print(f"  ✓ 打亂資料（buffer={min(1000, len(image_paths))}）")

        # 步驟5: 資料增強
        dataset = dataset.map(
            apply_augmentation,
            num_parallel_calls=AUTOTUNE
        )
        print(f"  ✓ 應用資料增強")

    # 步驟6: 批次化
    dataset = dataset.batch(batch_size, drop_remainder=is_training)
    print(f"  ✓ 批次化（drop_remainder={is_training}）")

    # 步驟7: 預取
    dataset = dataset.prefetch(AUTOTUNE)
    print(f"  ✓ 預取下一批資料")

    # 步驟8: 重複（訓練集無限循環）
    if is_training:
        dataset = dataset.repeat()
        print(f"  ✓ 無限重複（用於訓練）")

    return dataset

# 建立三個資料集
print("\n" + "="*70)
print("🏗️ 建立訓練/驗證/測試資料集...")
print("="*70)

train_dataset = create_dataset(
    train_images, train_masks,
    is_training=True,
    batch_size=BATCH_SIZE
)

val_dataset = create_dataset(
    val_images, val_masks,
    is_training=False,
    batch_size=BATCH_SIZE
)

test_dataset = create_dataset(
    test_images, test_masks,
    is_training=False,
    batch_size=BATCH_SIZE
)

# 計算訓練步數
steps_per_epoch = len(train_images) // BATCH_SIZE
validation_steps = len(val_images) // BATCH_SIZE
test_steps = len(test_images) // BATCH_SIZE

print("\n" + "="*70)
print("📊 資料集資訊摘要:")
print("="*70)

summary_data = {
    '資料集': ['訓練集', '驗證集', '測試集'],
    '樣本數量': [len(train_images), len(val_images), len(test_images)],
    '批次數量': [steps_per_epoch, validation_steps, test_steps],
    '批次大小': [BATCH_SIZE, BATCH_SIZE, BATCH_SIZE],
    '資料增強': ['✓', '✗', '✗'],
    '打亂': ['✓', '✗', '✗'],
    '重複': ['✓', '✗', '✗']
}

df_summary = pd.DataFrame(summary_data)
print(df_summary.to_string(index=False))

print(f"\n🔢 訓練配置:")
print(f"  每輪步數 (steps_per_epoch): {steps_per_epoch}")
print(f"  驗證步數 (validation_steps): {validation_steps}")
print(f"  測試步數 (test_steps): {test_steps}")

# 測試資料管線
print("\n" + "="*70)
print("🧪 測試資料管線...")
print("="*70)

# 從訓練集取一個批次
sample_batch_images, sample_batch_masks = next(iter(train_dataset))

print(f"\n批次資訊:")
print(f"  影像批次形狀: {sample_batch_images.shape}")
print(f"  遮罩批次形狀: {sample_batch_masks.shape}")
print(f"  影像數據類型: {sample_batch_images.dtype}")
print(f"  遮罩數據類型: {sample_batch_masks.dtype}")
print(f"  影像值範圍: [{sample_batch_images.numpy().min():.3f}, {sample_batch_images.numpy().max():.3f}]")
print(f"  遮罩值範圍: [{sample_batch_masks.numpy().min():.3f}, {sample_batch_masks.numpy().max():.3f}]")

# 視覺化一個批次
print("\n📊 視覺化訓練批次（前4個樣本）...")

fig, axes = plt.subplots(4, 2, figsize=(10, 16))

for i in range(min(4, BATCH_SIZE)):
    # 取出單個樣本
    img = sample_batch_images[i].numpy()
    mask = sample_batch_masks[i].numpy()

    # 反正規化影像
    img_display = img * dataset_std + dataset_mean
    img_display = np.clip(img_display, 0, 1)

    # 顯示影像
    axes[i, 0].imshow(img_display)
    axes[i, 0].set_title(f'批次樣本 {i+1} - 增強後影像', fontsize=11, fontweight='bold')
    axes[i, 0].axis('off')

    # 顯示遮罩
    im = axes[i, 1].imshow(mask[:, :, 0], cmap='jet', vmin=0, vmax=1)
    axes[i, 1].set_title(f'批次樣本 {i+1} - 對應遮罩', fontsize=11, fontweight='bold')
    axes[i, 1].axis('off')

    if i == 0:
        plt.colorbar(im, ax=axes[i, 1], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

# 測試迭代速度
print("\n" + "="*70)
print("⚡ 測試資料管線速度...")
print("="*70)

import time

# 測試讀取10個批次的時間
print(f"\n讀取 10 個批次...")
start_time = time.time()

for i, (images, masks) in enumerate(train_dataset.take(10)):
    pass

elapsed_time = time.time() - start_time

print(f"  總時間: {elapsed_time:.2f} 秒")
print(f"  平均每批次: {elapsed_time/10:.3f} 秒")
print(f"  吞吐量: {(BATCH_SIZE * 10) / elapsed_time:.1f} 樣本/秒")

print("\n💡 效能提示:")
print("  - 使用 cache() 可避免重複讀取磁碟")
print("  - 使用 prefetch() 可在 GPU 計算時預先準備資料")
print("  - 使用 num_parallel_calls 可平行處理資料")

print("\n" + "="*70)
print("✅ Dataset 管線建立完成！")
print("="*70)

In [ ]:
# ==================== Cell 13: 生成前處理報告與儲存設定 ====================
print("📋 生成資料前處理完整報告...")
print("="*70)

# 收集所有前處理資訊
preprocessing_report = {
    'dataset_info': {
        'total_samples': len(final_images),
        'train_samples': len(train_images),
        'val_samples': len(val_images),
        'test_samples': len(test_images),
        'image_size': IMAGE_SIZE,
        'num_classes': NUM_CLASSES,
        'unique_labels': unique_labels,
    },
    'normalization': {
        'method': 'dataset_statistics',
        'mean_rgb': dataset_mean.tolist(),
        'std_rgb': dataset_std.tolist(),
    },
    'augmentation': AUGMENTATION_CONFIG,
    'class_weights': class_weight_dict,
    'training_config': {
        'batch_size': BATCH_SIZE,
        'steps_per_epoch': steps_per_epoch,
        'validation_steps': validation_steps,
        'test_steps': test_steps,
    },
    'class_distribution': {
        'pixel_counts': dict(pixel_counts),
        'occurrences': dict(occurrences),
        'imbalance_ratio': float(imbalance_ratio),
    }
}

# 顯示完整報告
print("\n" + "="*70)
print("📊 資料前處理完整報告")
print("="*70)

print("\n【1. 資料集資訊】")
print(f"  總樣本數: {preprocessing_report['dataset_info']['total_samples']}")
print(f"  訓練集: {preprocessing_report['dataset_info']['train_samples']} ({len(train_images)/len(final_images)*100:.1f}%)")
print(f"  驗證集: {preprocessing_report['dataset_info']['val_samples']} ({len(val_images)/len(final_images)*100:.1f}%)")
print(f"  測試集: {preprocessing_report['dataset_info']['test_samples']} ({len(test_images)/len(final_images)*100:.1f}%)")
print(f"  影像尺寸: {IMAGE_SIZE}×{IMAGE_SIZE}")
print(f"  類別數量: {NUM_CLASSES}")

print("\n【2. 正規化參數】")
print(f"  方法: 資料集統計量正規化")
print(f"  RGB 均值: {dataset_mean}")
print(f"  RGB 標準差: {dataset_std}")

print("\n【3. 資料增強配置】")
for key, value in AUGMENTATION_CONFIG.items():
    print(f"  {key}: {value}")

print("\n【4. 類別權重（前5個）】")
for i, (class_id, weight) in enumerate(list(class_weight_dict.items())[:5]):
    print(f"  類別 {class_id}: {weight:.4f}")
print(f"  ... 共 {len(class_weight_dict)} 個類別")

print("\n【5. 訓練配置】")
print(f"  批次大小: {BATCH_SIZE}")
print(f"  每輪步數: {steps_per_epoch}")
print(f"  驗證步數: {validation_steps}")
print(f"  測試步數: {test_steps}")

print("\n【6. 類別不平衡】")
print(f"  不平衡比例: {imbalance_ratio:.2f}:1")
if imbalance_ratio > 100:
    print(f"  ⚠️ 嚴重不平衡，已計算類別權重")
elif imbalance_ratio > 10:
    print(f"  ⚠️ 中度不平衡，建議使用類別權重")
else:
    print(f"  ✓ 相對平衡")

# 儲存為 JSON
import json

report_filename = 'preprocessing_report.json'
with open(report_filename, 'w', encoding='utf-8') as f:
    json.dump(preprocessing_report, f, indent=2, ensure_ascii=False)

print(f"\n💾 報告已儲存至: {report_filename}")

# 建立視覺化總結
print("\n" + "="*70)
print("📈 視覺化總結報告...")
print("="*70)

fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. 資料集分割
ax1 = fig.add_subplot(gs[0, 0])
sets = ['訓練集', '驗證集', '測試集']
sizes = [len(train_images), len(val_images), len(test_images)]
colors_pie = ['#4CAF50', '#2196F3', '#FF9800']
ax1.pie(sizes, labels=sets, colors=colors_pie, autopct='%1.1f%%', startangle=90)
ax1.set_title('資料集分割', fontsize=12, fontweight='bold')

# 2. 類別數量
ax2 = fig.add_subplot(gs[0, 1])
ax2.text(0.5, 0.7, f'{NUM_CLASSES}', ha='center', va='center',
         fontsize=60, fontweight='bold', color='#2196F3')
ax2.text(0.5, 0.3, '類別總數', ha='center', va='center',
         fontsize=14, fontweight='bold')
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')

# 3. 不平衡比例
ax3 = fig.add_subplot(gs[0, 2])
color = '#F44336' if imbalance_ratio > 100 else '#FF9800' if imbalance_ratio > 10 else '#4CAF50'
ax3.text(0.5, 0.7, f'{imbalance_ratio:.1f}:1', ha='center', va='center',
         fontsize=40, fontweight='bold', color=color)
ax3.text(0.5, 0.3, '類別不平衡比', ha='center', va='center',
         fontsize=14, fontweight='bold')
ax3.set_xlim(0, 1)
ax3.set_ylim(0, 1)
ax3.axis('off')

# 4. 類別像素分佈
ax4 = fig.add_subplot(gs[1, :])
labels_bar = [str(l) for l in unique_labels]
pixels_bar = [pixel_counts[l] for l in unique_labels]
bars = ax4.bar(labels_bar, pixels_bar, color='steelblue', edgecolor='black', alpha=0.7)
ax4.set_xlabel('類別 ID', fontsize=12)
ax4.set_ylabel('總像素數（對數刻度）', fontsize=12)
ax4.set_title('各類別像素分佈', fontsize=14, fontweight='bold')
ax4.set_yscale('log')
ax4.grid(axis='y', alpha=0.3, linestyle='--')

# 5. 訓練配置
ax5 = fig.add_subplot(gs[2, 0])
config_text = f"""
訓練配置

批次大小: {BATCH_SIZE}
每輪步數: {steps_per_epoch}
驗證步數: {validation_steps}
影像尺寸: {IMAGE_SIZE}×{IMAGE_SIZE}
"""
ax5.text(0.5, 0.5, config_text, ha='center', va='center',
         fontsize=11, family='monospace',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
ax5.set_xlim(0, 1)
ax5.set_ylim(0, 1)
ax5.axis('off')

# 6. 正規化參數
ax6 = fig.add_subplot(gs[2, 1])
norm_text = f"""
正規化參數

方法: Dataset Statistics

RGB 均值:
R: {dataset_mean[0]:.4f}
G: {dataset_mean[1]:.4f}
B: {dataset_mean[2]:.4f}

RGB 標準差:
R: {dataset_std[0]:.4f}
G: {dataset_std[1]:.4f}
B: {dataset_std[2]:.4f}
"""
ax6.text(0.5, 0.5, norm_text, ha='center', va='center',
         fontsize=10, family='monospace',
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax6.set_xlim(0, 1)
ax6.set_ylim(0, 1)
ax6.axis('off')

# 7. 增強配置
ax7 = fig.add_subplot(gs[2, 2])
aug_text = "資料增強配置\n\n"
for key, value in AUGMENTATION_CONFIG.items():
    short_key = key.replace('_prob', '').replace('_', ' ').title()
    aug_text += f"{short_key}: {value}\n"
ax7.text(0.5, 0.5, aug_text, ha='center', va='center',
         fontsize=10, family='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))
ax7.set_xlim(0, 1)
ax7.set_ylim(0, 1)
ax7.axis('off')

plt.suptitle('DeepFashion 資料前處理完整報告', fontsize=16, fontweight='bold', y=0.98)
plt.savefig('preprocessing_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💾 視覺化報告已儲存至: preprocessing_summary.png")

# 顯示使用說明
print("\n" + "="*70)
print("📝 模型訓練時的使用方式")
print("="*70)

usage_code = """
# 在模型訓練時使用以下配置:

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=NUM_CLASSES)]
)

history = model.fit(
    train_dataset,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    class_weight=class_weight_dict,  # 使用類別權重
    callbacks=[...]
)

# 評估模型
test_results = model.evaluate(test_dataset, steps=test_steps)
"""

print(usage_code)

print("\n" + "="*70)
print("✅ 所有資料前處理完成！")
print("="*70)

print("\n📦 可用的變數:")
print("  - train_dataset: 訓練資料集")
print("  - val_dataset: 驗證資料集")
print("  - test_dataset: 測試資料集")
print("  - NUM_CLASSES: 類別數量")
print("  - class_weight_dict: 類別權重字典")
print("  - steps_per_epoch: 每輪訓練步數")
print("  - validation_steps: 驗證步數")
print("  - dataset_mean: 資料集均值")
print("  - dataset_std: 資料集標準差")

print("\n🎯 下一步: 建立並訓練模型！")